### TO-DO

- 네이버 쇼핑에서 원하는 검색어를 입력하여 나온 상품들의 5페이지까지의 상품들에 대한 정보를 가져옵니다. (~ 200개)

In [7]:
# 라이브러리 정의
import requests
import bs4
import pandas as pd
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
import time
from tqdm.notebook import tqdm
import warnings
warnings.simplefilter('ignore')

In [4]:
query = "삼성"

titles = []
prices = []
review_counts = []
buy_counts = []
published_dates = []
favorites = []

driver = Chrome()

for page_no in tqdm(range(1, 6)):

    page_url = f"https://search.shopping.naver.com/search/all?frm=NVSHATC&origQuery={query}&pagingIndex={page_no}&pagingSize=40&productSet=total&query={query}&sort=rel&timestamp=&viewType=list"
    driver.get(page_url)
    time.sleep(1)

    for scroll_down in range(7):
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(0.5)

  0%|          | 0/5 [00:00<?, ?it/s]

In [9]:
query = "삼성"

titles = []
prices = []
review_counts = []
buy_counts = []
published_dates = []
favorites = []

driver = Chrome()

for page_no in tqdm(range(1, 6)):

    page_url = f"https://search.shopping.naver.com/search/all?frm=NVSHATC&origQuery={query}&pagingIndex={page_no}&pagingSize=40&productSet=total&query={query}&sort=rel&timestamp=&viewType=list"
    driver.get(page_url)
    time.sleep(1)

    for scroll_down in range(7):
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(0.5)

    list_basis = driver.find_element(By.CLASS_NAME, "basicList_list_basis__uNBZx")
    # list_basis = driver.find_element_by_class_name("list_basis")    
    # item_list = list_basis.find_elements_by_class_name('basicList_inner__xCM3J')
    item_list = list_basis.find_elements(By.CLASS_NAME, 'product_inner__gr8QR')    

    items = []
    for i in tqdm(range(len(item_list))):
        item = item_list[i]
        title = item.find_element(By.CLASS_NAME, 'product_title__Mmw2K')
        # title = item.find_element_by_class_name('basicList_title__VfX3c')
        titles.append(title.text)

        price = item.find_element(By.CLASS_NAME, "price_num__S2p_v").text[:-1].replace(',', '')
        # price = item.find_element_by_class_name("price_num__S2p_v").text[:-1].replace(',', '')
        prices.append(price)

        footer = item.find_element(By.CLASS_NAME, 'product_etc_box__ElfVA')#.text
        # footer = item.find_element_by_class_name('basicList_etc_box__5lkgg')#.text
        # reviews = footer.find_elements_by_class_name('basicList_num__sfz3h')
        reviews = footer.find_elements(By.CLASS_NAME, 'product_grade__IzyU3')
        
        footer_text = footer.text
        try:
            if "구매건수" in footer_text:
                review_counts.append(int(reviews[0].text.replace(',', '')))
                buy_counts.append(int(reviews[1].text.replace(',', '')))
                favorites.append(int(reviews[2].text.replace(',', '')))
                date = footer.find_elements(By.TAG_NAME, 'span')[0].text[4:]
                published_dates.append(date)
            else:
                favorites.append(int(reviews[1].text.replace(',', '')))
                review_counts.append(int(reviews[0].text.replace(',', '')))
                date = footer.find_elements(By.CLASS_NAME, 'product_etc__LGVaW')[1].text[4:]
                published_dates.append(date)

        except IndexError: ## 리뷰가 아직 충분하지 않아서, 정보가 안뜨는 케이스
            review_counts.append(0)
            favorites.append(float(reviews[0].text.replace('별점\n', '')))
            date = footer.find_elements(By.TAG_NAME, 'span')[0].text[4:]
            published_dates.append(date)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

IndexError: list index out of range

In [16]:
print(len(titles), len(prices), len(review_counts), len(published_dates), len(favorites))
print(favorites)

81 81 81 80 80
[4.91, 4.9, 4.86, 4.92, 4.89, 4.85, 4.89, 4.89, 4.89, 4.84, 4.95, 4.89, 4.91, 4.93, 4.88, 4.84, 4.89, 4.92, 4.87, 4.72, 4.87, 4.86, 4.81, 4.86, 4.83, 4.97, 4.94, 4.92, 4.86, 4.78, 4.93, 4.86, 4.88, 4.86, 4.79, 4.89, 4.94, 4.86, 4.9, 4.92, 5.0, 4.87, 4.81, 5.0, 4.96, 4.84, 4.76, 4.93, 4.85, 4.88, 4.71, 4.85, 4.77, 4.85, 4.85, 4.85, 4.88, 4.9, 4.86, 4.86, 4.89, 4.87, 5.0, 4.8, 4.9, 4.88, 4.89, 4.88, 4.85, 4.94, 4.89, 4.86, 4.91, 4.89, 4.89, 4.81, 4.79, 4.93, 4.67, 4.93]


In [11]:
result = pd.DataFrame({"제품명" : titles,
                       "가격" : prices,
                       "리뷰수" : review_counts,
                       "등록일" : published_dates,
                       "찜하기" : favorites})
result

ValueError: All arrays must be of the same length

In [ ]:
result.to_excel(f"naver_shopping({query}).xlsx", index=False)